In [1]:
import math
import cv2
import numpy as np
import matplotlib.pyplot as plt
import import_ipynb
import matplotlib.patches as patches
import os
import torch
import torchvision.transforms as transforms
import skimage

from options.generate_options import GenerateOptions
from data.data_loader import CreateDataLoader
from models.models import create_model
from util.visualizer import Visualizer
from util.util import save_image
from PIL import Image

In [2]:
precision = 'fp32'
ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd', model_math=precision)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')
ssd_model = ssd_model.to('cuda')
ssd_model.eval()

Using cache found in /home/ashura1234/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
Using cache found in /home/ashura1234/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


SSD300(
  (feature_extractor): ResNet(
    (feature_extractor): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplac

In [3]:
def SSD(tensor, ssd_model):
    with torch.no_grad():
        detections = ssd_model(tensor)
    results = utils.decode_results(detections)
    best_results = [utils.pick_best(results, 0.40) for results in results]
    classes_to_labels = utils.get_coco_object_dictionary()
    person_counter = 0
    for image_idx in range(len(best_results)):
#         fig, ax = plt.subplots(1, figsize=(7,7))
        # Show original, denormalized image...
        image = tensor[image_idx] / 2 + 0.5
#         ax.imshow(image)
        # ...with detections
        bboxes, classes, confidences = best_results[image_idx]
        for idx in range(len(bboxes)):
#             left, bot, right, top = bboxes[idx]
#             x, y, w, h = [val * 300 for val in [left, bot, right - left, top - bot]]
#             rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
#             ax.add_patch(rect)
#             ax.text(x, y, "{} {:.0f}%".format(classes_to_labels[classes[idx] - 1], confidences[idx]*100), bbox=dict(facecolor='white', alpha=0.5))
            if classes[idx] == 1:
                person_counter += 1
#         plt.show()
    return person_counter

In [4]:
def load_PIL_image(PIL_image):
    """Code from Loading_Pretrained_Models.ipynb - a Caffe2 tutorial"""
    img = skimage.img_as_float(PIL_image)
    if len(img.shape) == 2:
        img = np.array([img, img, img]).swapaxes(0,2)
    return img

def prepare_input_from_PIL(PIL_img):
    img = load_PIL_image(PIL_img)
    img = utils.rescale(img, 300, 300)
    img = utils.crop_center(img, 300, 300)
    img = utils.normalize(img)

    return img

In [28]:
def is_longview(img, ssd_model) -> bool:
    img = cv2.resize(img, (455, 256))
    cv2.imshow("input", img)
    height, width, _ = img.shape
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    thres = 0.75
    h_lower = 30
    h_upper = 90
    
    field_mask = cv2.inRange(hsv, (h_lower, 0, 50), (h_upper, 255,255))
    persons = 5
    is_field = (np.sum(field_mask)/(width*height)/255 > thres)
#     if is_field:
#         inputs = [prepare_input_from_PIL(img)]
#         tensor = utils.prepare_tensor(inputs, precision == 'fp16').to('cuda')
#         persons = SSD(tensor, ssd_model)
    
    return is_field and persons > 3

In [29]:
opt = GenerateOptions()

opt.isTrain = False
opt.dataroot = "./datasets/generate"
opt.batchSize = 1
opt.loadSize = 256
opt.fineSize = 256
opt.input_nc = 3
opt.output_nc = 1
opt.ngf = 64
opt.ndf = 64
opt.which_model_netD = "basic"
opt.which_model_netG = "unet_256"
opt.n_layers_D = 3
opt.gpu_ids = [0]
opt.name = "soccer_seg_detection_pix2pix"
opt.dataset_mode = "single"
opt.model = "generate"
opt.which_direction = "AtoB"
opt.nThreads = 1
opt.checkpoints_dir = "./checkpoints"
opt.norm = "batch"
opt.serial_batches = True
opt.display_winsize = 256
opt.display_id = 1
opt.display_port = 8097
opt.no_dropout = True
opt.max_dataset_size = float("inf")
opt.resize_or_crop = 'resize_and_crop'
opt.no_flip = True
opt.init_type = 'normal'

opt.continue_train = False
opt.ntest = float("inf")
opt.results_dir = './'
opt.aspect_ratio = 1.0
opt.phase = "generate"
opt.which_epoch = "latest"
opt.how_many = 1
opt.output_dir = './'



# data_loader = CreateDataLoader(opt)
# dataset = data_loader.load_data()
model = create_model(opt)
# visualizer = Visualizer(opt)

counter = 1

generate
initialization method [normal]
initialization method [normal]
---------- Networks initialized -------------
UnetGenerator(
  (model): UnetSkipConnectionBlock(
    (model): Sequential(
      (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): UnetSkipConnectionBlock(
        (model): Sequential(
          (0): LeakyReLU(negative_slope=0.2, inplace=True)
          (1): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
          (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (3): UnetSkipConnectionBlock(
            (model): Sequential(
              (0): LeakyReLU(negative_slope=0.2, inplace=True)
              (1): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
              (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (3): UnetSkipConnectionBlock(
                (model): S

In [30]:
def get_transform(opt):
    transform_list = []
    if opt.resize_or_crop == 'resize_and_crop':
        osize = [opt.loadSize, opt.loadSize]
        transform_list.append(transforms.Scale(osize, Image.BICUBIC))
        transform_list.append(transforms.RandomCrop(opt.fineSize))
    elif opt.resize_or_crop == 'crop':
        transform_list.append(transforms.RandomCrop(opt.fineSize))
    elif opt.resize_or_crop == 'scale_width':
        transform_list.append(transforms.Lambda(
            lambda img: __scale_width(img, opt.fineSize)))
    elif opt.resize_or_crop == 'scale_width_and_crop':
        transform_list.append(transforms.Lambda(
            lambda img: __scale_width(img, opt.loadSize)))
        transform_list.append(transforms.RandomCrop(opt.fineSize))

    if opt.isTrain and not opt.no_flip:
        transform_list.append(transforms.RandomHorizontalFlip())

    transform_list += [transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5, 0.5, 0.5))]
    return transforms.Compose(transform_list)

In [31]:
def GAN_line_detection(input_img, counter):
    model.set_input(input_img)
    model.generate()
    visuals = model.get_current_visuals()    
    print('%04d: process image... ' % (counter))
    img = np.array(input_img['A'].squeeze())
    h = img.shape[1]
    w = img.shape[2]
    img = cv2.resize(img[:, :, int(w/2-h/2):int(w/2+h/2)], (256,256))
    
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    kernel2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
    
    output = cv2.cvtColor(visuals["fake_D"], cv2.COLOR_RGB2GRAY)
#     output = cv2.GaussianBlur(output,(5,5),10)
#     output = cv2.morphologyEx(output, cv2.MORPH_CLOSE, kernel2)
    
#     canny = cv2.Canny(output, 200, 120, apertureSize=3)
    
#     edges = cv2.dilate(canny,kernel1,iterations = 1)
#     edges = cv2.erode(edges,kernel1,iterations = 1)
#     edges = cv2.dilate(edges,kernel2,iterations = 1)
#     edges = cv2.erode(edges,kernel1,iterations = 1)
    
#     canny = cv2.Canny(edges, 200, 120, apertureSize=3)
    
#     edges = cv2.dilate(canny,kernel1,iterations = 1)
#     edges = cv2.erode(edges,kernel1,iterations = 1)
#     edges = cv2.dilate(edges,kernel2,iterations = 1)
#     edges = cv2.erode(edges,kernel1,iterations = 1)
    
#     lines = cv2.HoughLinesP(edges, rho = 1,theta = 1*np.pi/180,threshold = 10, minLineLength = 0,maxLineGap = 20)
#     if lines is not None:
#         print(output.shape)
#         for line in lines:
#             x1, y1, x2, y2 = line[0]
#             img = cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 1)
            
    cv2.imshow("Result", output)
#     f = plt.figure(1)
#     f.set_figheight(15)
#     f.set_figwidth(15)
    
#     plt.subplot(231)
#     plt.imshow(visuals["real_A"])

#     plt.subplot(232)
#     plt.imshow(visuals["fake_B"])
    
#     plt.subplot(233)
#     plt.imshow(visuals["fake_D"])
    
#     plt.subplot(234)
#     plt.imshow(canny, cmap='gray')
    
#     plt.subplot(235)
#     plt.imshow(edges, cmap='gray')
    
#     plt.subplot(236)
#     plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    
#     plt.show()

In [34]:
cap = cv2.VideoCapture("./videos/England v Croatia Smaller Clip 2.mp4")
counter = 0
para = None
para_ret = None
while(True):
    counter += 1
    ret, frame = cap.read()
    if frame is None:
        break
    if is_longview(frame, ssd_model):
        img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        w, h = img.size
        box = (int(w/2-h/2), 0, int(w/2+h/2), h)
        img = img.crop(box)
#         plt.imshow(img)
        transform = get_transform(opt)
        input_img = transform(img)
        input_img = {'A':input_img.unsqueeze(0), 'A_paths': ""}
        GAN_line_detection(input_img, counter)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()



0224: process image... 
0225: process image... 
0226: process image... 
0227: process image... 
0228: process image... 
0229: process image... 
0230: process image... 
0231: process image... 
0232: process image... 
0233: process image... 
0234: process image... 
0235: process image... 
0236: process image... 
0237: process image... 
0238: process image... 
0239: process image... 
0240: process image... 
0241: process image... 
0242: process image... 
0243: process image... 
0244: process image... 
0245: process image... 
0246: process image... 
0247: process image... 
0248: process image... 
0249: process image... 
0250: process image... 
0251: process image... 
0252: process image... 
0253: process image... 
0254: process image... 
0255: process image... 
0256: process image... 
0257: process image... 
0258: process image... 
0259: process image... 
0260: process image... 
0261: process image... 
0262: process image... 
0263: process image... 
0264: process image... 
0265: process im

1292: process image... 
1293: process image... 
1294: process image... 
1295: process image... 
1296: process image... 
1297: process image... 
1298: process image... 
1299: process image... 
1300: process image... 
1301: process image... 
1302: process image... 
1303: process image... 
1304: process image... 
1305: process image... 
1306: process image... 
1307: process image... 
1308: process image... 
1309: process image... 
1310: process image... 
1311: process image... 
1312: process image... 
1313: process image... 
1314: process image... 
1315: process image... 
1316: process image... 
1317: process image... 
1318: process image... 
1319: process image... 
1320: process image... 
1321: process image... 
1322: process image... 
1323: process image... 
1324: process image... 
1325: process image... 
1326: process image... 
1327: process image... 
1328: process image... 
1329: process image... 
1330: process image... 
1331: process image... 
1332: process image... 
1333: process im

1638: process image... 
1639: process image... 
1640: process image... 
1641: process image... 
1642: process image... 
1643: process image... 
1644: process image... 
1645: process image... 
1646: process image... 
1647: process image... 
1648: process image... 
1649: process image... 
1650: process image... 
1651: process image... 
1652: process image... 
1653: process image... 
1654: process image... 
1655: process image... 
1656: process image... 
1657: process image... 
1658: process image... 
1659: process image... 
1660: process image... 
1661: process image... 
1662: process image... 
1663: process image... 
1664: process image... 
1665: process image... 
1666: process image... 
1667: process image... 
1668: process image... 
1669: process image... 
1670: process image... 
1671: process image... 
1672: process image... 
1673: process image... 
1674: process image... 
1675: process image... 
1676: process image... 
1677: process image... 
1678: process image... 
1679: process im

2067: process image... 
2068: process image... 
2069: process image... 
2070: process image... 
2071: process image... 
2072: process image... 
2073: process image... 
2074: process image... 
2075: process image... 
2076: process image... 
2077: process image... 
2078: process image... 
2079: process image... 
2080: process image... 
2081: process image... 
2082: process image... 
2083: process image... 
2084: process image... 
2085: process image... 
2086: process image... 
2087: process image... 
2088: process image... 
2089: process image... 
2090: process image... 
2091: process image... 
2092: process image... 
2093: process image... 
2094: process image... 
2095: process image... 
2096: process image... 
2097: process image... 
2098: process image... 
2099: process image... 
2100: process image... 
2101: process image... 
2102: process image... 
2103: process image... 
2104: process image... 
2105: process image... 
2106: process image... 
2107: process image... 
2108: process im

2411: process image... 
2412: process image... 
2413: process image... 
2414: process image... 
2415: process image... 
2416: process image... 
2417: process image... 
2418: process image... 
2419: process image... 
2420: process image... 
2421: process image... 
2422: process image... 
2423: process image... 
2424: process image... 
2425: process image... 
2426: process image... 
2427: process image... 
2428: process image... 
2429: process image... 
2430: process image... 
2431: process image... 
2432: process image... 
2433: process image... 
2434: process image... 
2435: process image... 
2436: process image... 
2437: process image... 
2438: process image... 
2439: process image... 
2440: process image... 
2441: process image... 
2442: process image... 
2443: process image... 
2444: process image... 
2445: process image... 
2446: process image... 
2447: process image... 
2448: process image... 
2449: process image... 
2450: process image... 
2451: process image... 
2452: process im

2928: process image... 
2929: process image... 
2930: process image... 
2931: process image... 
2932: process image... 
2933: process image... 
2934: process image... 
2935: process image... 
2936: process image... 
2937: process image... 
2938: process image... 
2939: process image... 
2940: process image... 
2941: process image... 
2942: process image... 
2943: process image... 
2944: process image... 
2945: process image... 
2946: process image... 
2947: process image... 
2948: process image... 
2949: process image... 
2950: process image... 
2951: process image... 
2952: process image... 
2953: process image... 
2954: process image... 
2955: process image... 
2956: process image... 
2957: process image... 
2958: process image... 
2959: process image... 
2960: process image... 
2961: process image... 
2962: process image... 
2963: process image... 
2964: process image... 
2965: process image... 
2966: process image... 
2967: process image... 
2968: process image... 
2969: process im

In [10]:
def GAN_top_hat_line_detection(input_img, counter):
    model.set_input(input_img)
    model.generate()
    visuals = model.get_current_visuals()    
    img_path = model.get_image_paths()
    print('%04d: process image... %s' % (counter, img_path))
    counter += 1
    img = cv2.imread(img_path)
    h = img.shape[0]
    w = img.shape[1]
    img = cv2.resize(img[:, int(w/2-h/2):int(w/2+h/2), :], (256,256))
    mask = cv2.cvtColor(visuals["fake_B"], cv2.COLOR_BGR2GRAY)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    filterSize =(3, 3) 
#     kernel = cv2.getStructuringElement(cv2.MORPH_RECT,  
                                       filterSize) 
    tophat_img = cv2.morphologyEx(gray,  
                              cv2.MORPH_TOPHAT, 
                              kernel)
    edges = np.bitwise_and(tophat_img, mask)
    
    lines = cv2.HoughLinesP(edges, rho = 1,theta = 1*np.pi/180,threshold = 10, minLineLength = 0,maxLineGap = 20)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            img = cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 1)
    
    f = plt.figure(1)
    f.set_figheight(15)
    f.set_figwidth(15)
    
    plt.subplot(231)
    plt.imshow(visuals["real_A"])

    plt.subplot(232)
    plt.imshow(mask, cmap='gray')
    
    plt.subplot(233)
    plt.imshow(gray, cmap='gray')
    
    plt.subplot(234)
    plt.imshow(tophat_img, cmap='gray')
    
    plt.subplot(235)
    plt.imshow(edges, cmap='gray')
    
    plt.subplot(236)
    plt.imshow(img)
    
    plt.show()

In [11]:
# GAN_top_hat_line_detection(input_img, counter)